Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [600]:
from random import choices,randint, random
from collections import namedtuple
import logging

import lab9_lib

In [601]:
NUM_LOCI = 1000s
POPULATION_SIZE = 50
OFFSPRING_SIZE = 20
INSTANCES = 2

NUM_GENERATIONS = 100

In [602]:
fitness = lab9_lib.make_problem(INSTANCES)
for n in range(POPULATION_SIZE):
    ind = choices([0, 1], k=NUM_LOCI)
    # print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

0


Initialize population

In [603]:
class Individual:
    def __init__(self, genome, fitness):
        self.genome = genome
        self.fitness = fitness

def initialize_population(POPULATION_SIZE, NUM_LOCI):
    population = []

    for _ in range(POPULATION_SIZE):
        genome = tuple(choices([0, 1], k=NUM_LOCI))
        Fitness = fitness(genome)
        individual = Individual(genome=genome, fitness=Fitness)
        population.append(individual)

    return population


Cross over and mutation functions

In [604]:
def single_point_crossover(genome1, genome2):
    point = randint(0, len(genome1))
    return list(genome1[:point]) + list(genome2[point:])
    # return genome1[:point] + genome2[point:], genome2[:point] + genome1[point:]


def multiple_point_crossover(genome1, genome2, num_points):
    points = [randint(0, len(genome1)) for _ in range(num_points)]
    points.sort()

    offspring = []
    last_point = 0

    for point in points:
        offspring += genome1[last_point:point]
        genome1, genome2 = genome2, genome1  # Swap the parents
        last_point = point
    offspring += genome1[last_point:]
    return offspring


def mutate(genome):
    point = randint(0, NUM_LOCI - 1)
    return genome[:point] + (1 - genome[point],) + genome[point + 1 :]

def tournament(population, size=2):
    selected = choices(population, k=size)
    return max(selected, key=lambda j:j.fitness)

In [605]:
def ss_GA(population, offspring, size):
    population += offspring
    population = sorted(population, key=lambda i: i.fitness, reverse=True)[:size]

    return population

#algorithm based on the professor's repository

In [606]:

selection = ss_GA
best_fitness = 0
counter = 0

for g in range(NUM_GENERATIONS):
    offspring = list()
    
    currentfitness = 0

    for i in range(OFFSPRING_SIZE):
        if random() < 0.3:
            p = tournament(population)
            o = mutate(p.genome)
        else:
            p1 = tournament(population)
            p2 = tournament(population)
            o = single_point_crossover(p1.genome, p2.genome)
        f = fitness(o)

        if f > currentfitness: 
            currentfitness = f
        offspring.append(Individual(o, f))
    population = selection(population, offspring, POPULATION_SIZE)
    
    if currentfitness <= best_fitness:
        counter += 1
        
        if counter >= 100:
            print("break at generation n.", g, "fitness: ", best_fitness)
            break
        
    elif currentfitness > best_fitness:
        counter = 0

        best_fitness = currentfitness
        if best_fitness == 1:
            print("break at generation n.", g, "fitness: ", best_fitness)
            break
    
    print("Generation",g ,"counter:", counter, "best_fitness:", best_fitness, "difference:",best_fitness-currentfitness)

population += offspring
population = sorted(population, key=lambda i: i.fitness, reverse=True)[:POPULATION_SIZE]
print(population[0])
print(fitness.calls)


TypeError: can only concatenate list (not "tuple") to list